# CS130 - Assignment 3: Causal Inference

## Question 1: "Daughters"
*Read Section 5 (which is only about 1 page long!) of Iacus, King, Porro (2011), Multivariate Matching Methods That Are Monotonic Imbalance Bounding, JASA, V 106, N. 493, available here: https://gking.harvard.edu/files/gking/files/cem_jasa.pdf Don't worry about the "CEM" elements. Focus on the "daughters" case study.*

In [56]:
# Load necessary packages.
library(arm)
library(Matching)
library(rgenoud)
library("IRdisplay")

In [3]:
# Data for this case study is available in "foo" below.
foo <- read.csv(url("https://course-resources.minerva.kgi.edu/uploaded_files/mke/00089202-1711/daughters.csv"))

head(foo)
nrow(foo)
ncol(foo)

,X,year,congress,party,district,statenam,name,ngirls,nboys,totchi,⋯,reg3,reg4,reg5,reg6,reg7,reg8,reg9,Dems,Repubs,OthParty
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1997-1998,105,0,1,HAWAII,"ABERCROMBIE, NEIL",0,0,0,⋯,0,0,0,0,0,0,1,1,0,0
2,2,1997-1998,105,0,5,NEW YOR,"ACKERMAN, GARY L.",1,2,3,⋯,0,0,0,0,0,0,0,1,0,0
3,3,1997-1998,105,1,4,ALABAMA,"ADERHOLT, ROBERT B.",0,0,0,⋯,0,0,0,1,0,0,0,0,1,0
4,4,1997-1998,105,0,1,MAINE,"ALLEN, THOMAS H.",2,0,2,⋯,0,0,0,0,0,0,0,1,0,0
5,5,1997-1998,105,0,1,NEW JER,"ANDREWS, ROBERT E.",2,0,2,⋯,0,0,0,0,0,0,0,1,0,0
6,6,1997-1998,105,1,7,TEXAS,"ARCHER, W.R.",3,4,7,⋯,0,0,0,0,1,0,0,0,1,0


[1] 430

[1] 83

### PART (A)

#### Regression
*Before doing any matching, RUN A REGRESSION, with Y = nowtot, and the independent vars mentioned below: Dems, Repubs, Christian, age, srvlng, demvote, treatment  = hasgirls. Show the regression specification. Use the regression to estimate a treatment effect and confidence interval. Check the balance of this not-matched data set.*

In [12]:
lm1 <- lm(foo$nowtot ~ foo$Dems + foo$Repubs + foo$Christian +
         foo$age + foo$srvlng + foo$demvote + foo$hasgirls, data = foo)

summary(lm1)


Call:
lm(formula = foo$nowtot ~ foo$Dems + foo$Repubs + foo$Christian + 
    foo$age + foo$srvlng + foo$demvote + foo$hasgirls, data = foo)

Residuals:
    Min      1Q  Median      3Q     Max 
-56.028 -10.322  -1.517  11.208  69.642 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    38.6991    18.6306   2.077 0.038390 *  
foo$Dems       -8.1022    17.5861  -0.461 0.645238    
foo$Repubs    -55.1069    17.6340  -3.125 0.001901 ** 
foo$Christian -13.3961     3.7218  -3.599 0.000357 ***
foo$age         0.1260     0.1117   1.128 0.259938    
foo$srvlng     -0.2251     0.1355  -1.662 0.097349 .  
foo$demvote    87.5501     8.4847  10.319  < 2e-16 ***
foo$hasgirls   -0.4523     1.9036  -0.238 0.812322    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 17.19 on 422 degrees of freedom
Multiple R-squared:  0.7821,	Adjusted R-squared:  0.7784 
F-statistic: 216.3 on 7 and 422 DF,  p-value: < 2.2e-16


In [5]:
confint(lm1)

,2.5 %,97.5 %
(Intercept),2.07870870,75.31946777
foo$Dems,-42.66951687,26.46501755
foo$Repubs,-89.76834880,-20.44543598
foo$Christian,-20.71160536,-6.08052567
foo$age,-0.09351808,0.34542799
foo$srvlng,-0.49145173,0.04119929
foo$demvote,70.87248062,104.22768129
foo$hasgirls,-4.19406023,3.28952473


Before doing matching, we ran a regression as a first step to understanding the effect of having a daughter on congressional representation. The study itself would help understand congressional decision-making. Specifically, we look at the example of an agreement with the National Organization for Women (NOW).

The outcome we are measuring is the congressional voting score, Y = nowtot. The predictors are democratic/republican party, religion (Christian), age, srvlng, demvote with treatment as having girls. We want to figure out whether having daughters (hasgirls) is correlated with voting score (nowtot). 

As observed from regression summary lm1, the treatment effect is -0.4523, meaning there is a negative association between nowtot and hasgirls. Finally, the 95% confidence interval for hasgirls is [-4.19, 3.29]. Since the range between upper and lower bound is quite broad, we are unsure of what the coefficient estimate is. Thus, we cannot use regression alone to suggest the impact of having daughters on voting nature.

#### Genetic Matching

*Do genetic matching, using the draft code below. Use the same variables as in the regression above.*

*Summarize (in 5-15 sentences) the genetic matching procedure and results, including what you matched on, what you balanced on, and what your balance results were.*

*If you obtain high balance, consider rerunning with M = 2 or 3... If/when you are satisfied by balance achieved, then rerun Match() with Y included and obtain the treatment effect estimate, the standard error, and the confidence interval.*

In [65]:
# Checking current balance of a non-matched set.
mout <- Match(Tr = foo$hasgirls, X = lm1$fitted.values, M = 1, estimand = "ATT")
MatchBalance(foo$hasgirls ~ foo$Dems + foo$Repubs + foo$Christian + foo$age + foo$srvlng + foo$demvote, data = foo, match.out = mout, nboots = 250)


***** (V1) foo$Dems *****
                       Before Matching 	 	 After Matching
mean treatment........    0.45833 	 	    0.45833 
mean control..........    0.50847 	 	    0.46154 
std mean diff.........    -10.047 	 	   -0.64223 

mean raw eQQ diff.....   0.050847 	 	  0.0028409 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff.....          1 	 	          1 

mean eCDF diff........   0.025071 	 	  0.0014205 
med  eCDF diff........   0.025071 	 	  0.0014205 
max  eCDF diff........   0.050141 	 	  0.0028409 

var ratio (Tr/Co).....    0.98809 	 	    0.99897 
T-test p-value........    0.35571 	 	    0.31731 


***** (V2) foo$Repubs *****
                       Before Matching 	 	 After Matching
mean treatment........    0.53846 	 	    0.53846 
mean control..........    0.49153 	 	    0.53846 
std mean diff.........        9.4 	 	          0 

mean raw eQQ diff.....   0.042373 	 	          0 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff..... 

In [23]:
# Run genetic balance including dependent variable Y.
set.seed(2324)

genout1 <- GenMatch(Tr = foo$hasgirls, X = cbind(foo$Dems, foo$Repubs, foo$Christian, foo$age, foo$srvlng, foo$demvote), pop.size = 20, nboots=250,  M = 1, estimand = "ATT")



Sat Apr 16 17:37:11 2022
Domains:
 0.000000e+00   <=  X1   <=    1.000000e+03 
 0.000000e+00   <=  X2   <=    1.000000e+03 
 0.000000e+00   <=  X3   <=    1.000000e+03 
 0.000000e+00   <=  X4   <=    1.000000e+03 
 0.000000e+00   <=  X5   <=    1.000000e+03 
 0.000000e+00   <=  X6   <=    1.000000e+03 

Data Type: Floating Point
Operators (code number, name, population) 
	(1) Cloning........................... 	5
	(2) Uniform Mutation.................. 	2
	(3) Boundary Mutation................. 	2
	(4) Non-Uniform Mutation.............. 	2
	(5) Polytope Crossover................ 	2
	(6) Simple Crossover.................. 	2
	(7) Whole Non-Uniform Mutation........ 	2
	(8) Heuristic Crossover............... 	2
	(9) Local-Minimum Crossover........... 	0

SOFT Maximum Number of Generations: 100
Maximum Nonchanging Generations: 4
Population size       : 20
Convergence Tolerance: 1.000000e-03

Not Using the BFGS Derivative Based Optimizer on the Best Individual Each Generation.
Not Checkin

In [8]:
mout1 <- Match(Y = foo$nowtot, Tr = foo$hasgirls, X = cbind(foo$Dems, foo$Repubs, foo$Christian, foo$age, foo$srvlng, foo$demvote), Weight.matrix = genout1, M = 1, estimand = "ATT")

MatchBalance(foo$hasgirls ~ foo$Dems + foo$Repubs + foo$Christian + foo$age  + foo$srvlng + foo$demvote,
             match.out = mout1, nboots = 250)


***** (V1) foo$Dems *****
                       Before Matching 	 	 After Matching
mean treatment........    0.45833 	 	    0.45833 
mean control..........    0.50847 	 	    0.46154 
std mean diff.........    -10.047 	 	   -0.64223 

mean raw eQQ diff.....   0.050847 	 	  0.0032051 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff.....          1 	 	          1 

mean eCDF diff........   0.025071 	 	  0.0016026 
med  eCDF diff........   0.025071 	 	  0.0016026 
max  eCDF diff........   0.050141 	 	  0.0032051 

var ratio (Tr/Co).....    0.98809 	 	    0.99897 
T-test p-value........    0.35571 	 	    0.31731 


***** (V2) foo$Repubs *****
                       Before Matching 	 	 After Matching
mean treatment........    0.53846 	 	    0.53846 
mean control..........    0.49153 	 	    0.53846 
std mean diff.........        9.4 	 	          0 

mean raw eQQ diff.....   0.042373 	 	          0 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff..... 

In [9]:
# The Treatment Effect Estimate and The Standard Error.
summary(mout1)


Estimate...  1.0737 
AI SE......  2.213 
T-stat.....  0.48519 
p.val......  0.62754 

Original number of observations..............  430 
Original number of treated obs...............  312 
Matched number of observations...............  312 
Matched number of observations  (unweighted).  312 



In [10]:
# Find Confidence Interval.
lower <- mout1$est - 1.96*mout1$se
upper <- mout1$est + 1.96*mout1$se

print(lower)
print(upper)

         [,1]
[1,] -3.26369
         [,1]
[1,] 5.411126


First, we ran a matching procedure to check the current balance of the variable: the age variable had the lowest p-value before and after matching, corresponding to 0.002 and 2.22e-16, respectively. 

After, we ran genetic balance, including dependent variable Y. Here, we use the same independent variables as in regression, except we now match on Tr = hasgirls, the treatment variable. Procedure:

1. Run GenMatch() to obtain the best weights (scaling factors) on all the covariates.
2. Run Match() using obtained weights.
3. Check the balance as a result.

Looking at the output, we see that the minimum p-value before matching was 2.22e-16 for age. Now, the minimum was 0.31731 for variable dems. This is a significant improvement since the higher the p-value score on matching, the better balance we obtain. 

After matching 213 observations out of 430 available, we got the treatment effect as 1.0737 with a standard error of 2.213. TH result is not statistically significant given p-value is above the 0.05 threshold - 0.62754. The confidence interval is [-3.26, 5.41]. Because values are both above and below 0, we cannot even state whether there was an effect of having daughters on voting behavior. Confounding variables like having a son and a daughter likely influenced the result. Therefore, we will filter data to run the same analysis next. 

### PART (B)

#### Genetic Matching With Filtered Data
*Repeat everything  you've done for Part (A), including the regression, genetic algorithm, code, output, and 5-15 sentences, EXCEPT this time change the definition of treatment to cover at least 2 girls and no boys, and change the definition of control to cover at least 2 boys and no girls. Exclude all observations that don't meet these requirements. Be sure to explain (in a sentence or two) what you're doing with your new treatment and control definitions. Do your new definitions change anything?*

In [62]:
# Filter the data.
control <- subset(foo, ngirls >= 2 & nboys == 0)
treatment <- subset(foo, ngirls == 0 & nboys >=2)
foo2 <- rbind(control, treatment)
foo2

,X,year,congress,party,district,statenam,name,ngirls,nboys,totchi,⋯,reg3,reg4,reg5,reg6,reg7,reg8,reg9,Dems,Repubs,OthParty
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
4,4,1997-1998,105,0,1,MAINE,"ALLEN, THOMAS H.",2,0,2,⋯,0,0,0,0,0,0,0,1,0,0
5,5,1997-1998,105,0,1,NEW JER,"ANDREWS, ROBERT E.",2,0,2,⋯,0,0,0,0,0,0,0,1,0,0
12,12,1997-1998,105,1,10,NORTH C,"BALLENGER, CASS",3,0,3,⋯,0,0,1,0,0,0,0,0,1,0
21,21,1997-1998,105,0,30,CALIFOR,"BECERRA, XAVIER",2,0,2,⋯,0,0,0,0,0,0,1,1,0,0
22,22,1997-1998,105,0,25,TEXAS,"BENTSEN, KENNETH",2,0,2,⋯,0,0,0,0,1,0,0,1,0,0
24,24,1997-1998,105,0,26,CALIFOR,"BERMAN, HOWARD L.",2,0,2,⋯,0,0,0,0,0,0,1,1,0,0
34,34,1997-1998,105,1,8,OHIO,"BOEHNER, JOHN A.",2,0,2,⋯,1,0,0,0,0,0,0,0,1,0
45,45,1997-1998,105,0,13,OHIO,"BROWN, SHERROD",2,0,2,⋯,1,0,0,0,0,0,0,1,0,0
68,68,1997-1998,105,0,6,SOUTH C,"CLYBURN, JAMES E.",3,0,3,⋯,0,0,1,0,0,0,0,1,0,0


In [67]:
# Regression for filtered data.
lm2 <- lm(foo2$nowtot ~ foo2$Dems + foo2$Repubs + foo2$Christian +
         foo2$age + foo2$srvlng + foo2$demvote + foo2$hasgirls, data = foo2)

summary(lm2)


Call:
lm(formula = foo2$nowtot ~ foo2$Dems + foo2$Repubs + foo2$Christian + 
    foo2$age + foo2$srvlng + foo2$demvote + foo2$hasgirls, data = foo2)

Residuals:
    Min      1Q  Median      3Q     Max 
-51.636  -6.608   0.928   8.992  46.235 

Coefficients: (1 not defined because of singularities)
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -18.4332    16.3697  -1.126  0.26335    
foo2$Dems       49.9284     4.4002  11.347  < 2e-16 ***
foo2$Repubs          NA         NA      NA       NA    
foo2$Christian  -3.4303     7.7211  -0.444  0.65798    
foo2$age        -0.2558     0.2180  -1.173  0.24395    
foo2$srvlng      0.3908     0.2654   1.473  0.14461    
foo2$demvote    86.6631    17.9813   4.820 6.32e-06 ***
foo2$hasgirls   12.2925     3.5008   3.511  0.00072 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15.82 on 84 degrees of freedom
Multiple R-squared:  0.839,	Adjusted R-squared:  0.8275 
F-statistic: 72

In [68]:
confint(lm2)

,2.5 %,97.5 %
(Intercept),-50.9860577,14.1196514
foo2$Dems,41.1780715,58.6786775
foo2$Repubs,NA,NA
foo2$Christian,-18.7845865,11.9239371
foo2$age,-0.6891980,0.1776850
foo2$srvlng,-0.1369568,0.9185161
foo2$demvote,50.9052404,122.4209484
foo2$hasgirls,5.3308756,19.2542092


Here, we run the same regression as in part 1. The only difference is we filtered data to decrease confounding, thus, getting more precise results. By eliminating congress members who have both daughters and sons, we increased the hasgirls coefficient estimate - by 12.2925. It now shows a positive correlation between hasgirls and nowtot. Compared to a Part A result of nearly a zero, this is a significant improvement. The confidence interval [5.33, 19.25] now also includes values only above 0; it is a more precise estimate and gives a better understanding of the relationship between treatment and outcome variables. 

In [66]:
# Checking current balance of a non-matched set.
mout <- Match(Tr = foo2$hasgirls, X = cbind(foo2$Dems, foo2$Repubs, foo2$Christian, foo2$age, foo2$srvlng, foo2$demvote), M = 1, estimand = "ATT")
MatchBalance(foo2$hasgirls ~ foo2$Dems + foo2$Repubs + foo2$Christian + foo2$age + foo2$srvlng + foo2$demvote, data = foo2, match.out=mout, nboots=250)


***** (V1) foo2$Dems *****
                       Before Matching 	 	 After Matching
mean treatment........    0.61702 	 	    0.61702 
mean control..........    0.40909 	 	    0.61702 
std mean diff.........     42.317 	 	          0 

mean raw eQQ diff.....    0.20455 	 	          0 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff.....          1 	 	          0 

mean eCDF diff........    0.10397 	 	          0 
med  eCDF diff........    0.10397 	 	          0 
max  eCDF diff........    0.20793 	 	          0 

var ratio (Tr/Co).....    0.97609 	 	          1 
T-test p-value........    0.04806 	 	          1 


***** (V2) foo2$Repubs *****
                       Before Matching 	 	 After Matching
mean treatment........    0.38298 	 	    0.38298 
mean control..........    0.59091 	 	    0.38298 
std mean diff.........    -42.317 	 	          0 

mean raw eQQ diff.....    0.22727 	 	          0 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff....

In [36]:
# Run genetic balance including dependent variable Y.
genout2 <- GenMatch(Tr = foo2$hasgirls, X = cbind(foo2$Dems, foo2$Repubs, foo2$Christian, foo2$age, foo2$srvlng, foo2$demvote), pop.size = 20, nboots=250,  M = 1, estimand = "ATT")



Sat Apr 16 17:51:29 2022
Domains:
 0.000000e+00   <=  X1   <=    1.000000e+03 
 0.000000e+00   <=  X2   <=    1.000000e+03 
 0.000000e+00   <=  X3   <=    1.000000e+03 
 0.000000e+00   <=  X4   <=    1.000000e+03 
 0.000000e+00   <=  X5   <=    1.000000e+03 
 0.000000e+00   <=  X6   <=    1.000000e+03 

Data Type: Floating Point
Operators (code number, name, population) 
	(1) Cloning........................... 	5
	(2) Uniform Mutation.................. 	2
	(3) Boundary Mutation................. 	2
	(4) Non-Uniform Mutation.............. 	2
	(5) Polytope Crossover................ 	2
	(6) Simple Crossover.................. 	2
	(7) Whole Non-Uniform Mutation........ 	2
	(8) Heuristic Crossover............... 	2
	(9) Local-Minimum Crossover........... 	0

SOFT Maximum Number of Generations: 100
Maximum Nonchanging Generations: 4
Population size       : 20
Convergence Tolerance: 1.000000e-03

Not Using the BFGS Derivative Based Optimizer on the Best Individual Each Generation.
Not Checkin

In [39]:
mout2 <- Match(Y = foo2$nowtot, Tr = foo2$hasgirls, X = cbind(foo2$Dems, foo2$Repubs, foo2$Christian, foo2$age, foo2$srvlng, foo2$demvote), Weight.matrix = genout2, M = 1, estimand = "ATT")

MatchBalance(foo2$hasgirls ~ foo2$Dems + foo2$Repubs + foo2$Christian + foo2$age  + foo2$srvlng + foo2$demvote,
             match.out = mout2, nboots = 250)


***** (V1) foo2$Dems *****
                       Before Matching 	 	 After Matching
mean treatment........    0.61702 	 	    0.61702 
mean control..........    0.40909 	 	    0.61702 
std mean diff.........     42.317 	 	          0 

mean raw eQQ diff.....    0.20455 	 	          0 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff.....          1 	 	          0 

mean eCDF diff........    0.10397 	 	          0 
med  eCDF diff........    0.10397 	 	          0 
max  eCDF diff........    0.20793 	 	          0 

var ratio (Tr/Co).....    0.97609 	 	          1 
T-test p-value........    0.04806 	 	          1 


***** (V2) foo2$Repubs *****
                       Before Matching 	 	 After Matching
mean treatment........    0.38298 	 	    0.38298 
mean control..........    0.59091 	 	    0.38298 
std mean diff.........    -42.317 	 	          0 

mean raw eQQ diff.....    0.22727 	 	          0 
med  raw eQQ diff.....          0 	 	          0 
max  raw eQQ diff....

In [41]:
# The Treatment Effect Estimate and The Standard Error.
summary(mout2)


Estimate...  11.383 
AI SE......  4.0247 
T-stat.....  2.8283 
p.val......  0.0046797 

Original number of observations..............  91 
Original number of treated obs...............  47 
Matched number of observations...............  47 
Matched number of observations  (unweighted).  47 



In [42]:
# Find Confidence Interval.
lower <- mout2$est - 1.96*mout2$se
upper <- mout2$est + 1.96*mout2$se

print(lower)
print(upper)

         [,1]
[1,] 3.494597
         [,1]
[1,] 19.27136


The balance before matching increased too. The minimum p-values before and after matching were 0.04806 and 0.19537 for Reepubs and srvlng, respectively. 

After running genetic matching, including the independent variable Y, we matched 47 out of 91 observations, getting the minimum p-value after matching of 0.44246 for variable demvote. Thus, we achieved a higher balance than Part A, where the minimum was 0.31731 for variable dems. The treatment effect estimate became 11.383 with the standard error of 4.0247. The p-value in filtered data now showed that the result was statistically significant, given that 0.0046797 is less than the 0.05 threshold. The confidence interval also was more precise - [3.49, 19.27].

## Question 3

*Run a sensitivity test to hidden bias (e.g., a Rosenbaum sensitivity test) on your results in Question 2 Part (A) and Question 2 Part (B) and write a paragraph explaining what you find. Be sure to describe the degree of sensitivity to hidden bias.

Unfortunately, the library rbound no longer works with the current R version, meaning we cannot use the psens() function to run the Rosenbaum Sensitivity Test. Instead, we used the following resource https://carloscinelli.shinyapps.io/robustness_value/, where we input treatment estimate, standard error, and degrees of freedom to run sensitivity analysis for each parts A and B. 

First, if we believe that the obtained p-value is true, then we think there is no hidden bias = no unobserved confounding. This is highly unlikely, so our result is susceptible to hidden bias due to the p-value in part B being on edge with a 0.05 threshold which could increase when increasing gamma from 1 to 2.

**Part A**
R^2 of the treatment with the outcome - 0.03%
Robustness Value for the point estimate	- 1.82%
Robustness Value for the t-value - 0%

**Part B**
R^2 of the treatment with the outcome - 1.13%
Robustness Value for the point estimate	- 10.13%
Robustness Value for the t-value - 3.21%

After using the software, we get three values. R^2 tells how strong unobservable confounders must be to explain the result. In Part B, R^2 is much higher, meaning with such precise data filtering, the possibility of unobservable confounders is harder to explain. 

Both robustness values for point estimates and t-values explain the extent of how robust unobservable confounders should be to make treatment effect estimates zero. Again, we see that the second model performs much better. 

---